In [2]:
#imports
import spacy
import random
import numpy as np
import pickle
import pandas as pd
import itertools
import scipy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise
from sklearn.cluster import KMeans

In [3]:
with open("wordlist.pkl", "rb") as f:
    words = pickle.load(f)

In [4]:
with open("word_lexicals.pkl", "rb") as f:
    texts = pickle.load(f)

In [5]:
texts["twitter"]

['basant',
 'bloquer',
 'TechCrunch',
 'supprimer',
 'suivre',
 'Bieber',
 'diffuser',
 'hashtag',
 'PDG',
 'compte',
 'Justin',
 'outil',
 'twittosphère',
 'poster',
 'startup',
 'social',
 'réseau',
 'Vine',
 'tweet',
 'blog',
 'afficher',
 'circuler',
 'LinkedIn',
 'réseaux',
 'média',
 'followers',
 'Evan',
 'actualité',
 'Obama',
 'Katy',
 'fil',
 'censure',
 'abonné',
 'Pinterest',
 'internaute',
 'Instagram',
 'Jack',
 'publicité',
 'application',
 'formidable',
 'api',
 'message',
 'sociaux',
 'caractères',
 'Wikipédia',
 'fonctionnalité',
 'Perry',
 'YouTube',
 'racheté',
 'filtrer',
 'twitteur',
 'dollar',
 'twittos',
 'utilisateur',
 'Google',
 'glass',
 'retweeter',
 'notification',
 'follower',
 'plateforme',
 'Barack',
 'Soundcloud',
 'Facebook',
 'suppression',
 'onglet',
 'site',
 'Dorsey',
 'rumeur',
 'concision',
 'SMS',
 'certifié',
 'bourse',
 'web',
 'valorisation']

In [6]:
df = pd.read_excel("Lexique383.xlsb", engine='pyxlsb')

In [7]:
nlp = spacy.load("fr_core_news_lg")

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise

In [9]:
sample_words = random.sample(words, 2)

In [10]:
def calculate_distance(a, b, option="cosine"):
    if option == "euclidean":
        return np.linalg.norm(a-b)
    if option == "scalar":
        return a @ b
    if option == "cosine":
        return pairwise.cosine_similarity(a.reshape(1,-1), b.reshape(1,-1))[0][0]

In [11]:
def calculate_proximity_score(word_candidate, word_board):
    w1 = nlp.vocab.get_vector(str(word_candidate))
    w2 = nlp.vocab.get_vector(str(word_board))
    if word_candidate in texts[word_board]:
        return 1
    return calculate_distance(w1, w2)

In [12]:
def get_score_df(pos, neg, neu, assassin, indices, danger_coeff):
    
    all_vectors = []
    all_words_candidate = []
    for w in pos:
        for lexical_w in texts[w]:
            all_vectors.append(nlp.vocab.get_vector(lexical_w))
            all_words_candidate.append(lexical_w)
            
    pos_dist = [[calculate_proximity_score(word_candidate, word_board)for word_board in pos] for word_candidate in all_words_candidate]
    neg_dist = [[calculate_proximity_score(word_candidate, word_board)for word_board in neg] for word_candidate in all_words_candidate]
    neu_dist = [[calculate_proximity_score(word_candidate, word_board)for word_board in neu] for word_candidate in all_words_candidate]
    ass_dist = [calculate_proximity_score(word_candidate, assassin) for word_candidate in all_words_candidate]
    
    
    #build result df, containing candidate and their scores (positive and negative)
    df = pd.DataFrame(pos_dist, index=all_words_candidate)
    #compute a score for each row (a candidate)
    #df["score"] = df.apply(lambda x: np.product(x) / np.sum(x), axis=1)
    
    #divide by 2 to compare with our score function
    df["neg"] = np.max(neg_dist, axis=1) * danger_coeff
    if len(neu) > 0:
        df["neu"] = np.max(neu_dist, axis=1) 
    df["ass"] = np.array(ass_dist) * danger_coeff
    
    #remove words
    for w in np.concatenate([pos, indices]):
        if w.lower() in all_words_candidate:
            #stopwords
            df.drop(w, inplace=True)
    
    df.drop_duplicates(inplace=True)
    return df

In [13]:
def calculate_clue_score(clue, pos, agg, df, neu, vecteur_preference):
    scores = df.loc[df.index == clue]
    if len(neu) == 0:
        if scores[["neg", "ass"]].max(axis=1)[0] == 1:
            return -1, None
    else:
        if scores[["neg", "neu", "ass"]].max(axis=1)[0] == 1:
            return -1, None
    # Order scores by highest to lowest (highest is best)
    pos_scores = scores.iloc[:, :len(pos)].values[0]
    boundary = np.max(scores.iloc[:,len(pos):], axis=1).values[0]
    i = 0
    
    # Order scores by lowest to highest inner product with the clue.
    ss = sorted((s, i) for i, s in enumerate(pos_scores))
    groups = []
    groups_score = []
    for i in range(len(pos_scores)):
        group = ss[-(i+1):]
#         print('GROUPE: ', group)
        group_score = []
        for tpl in group:
            group_score.append(tpl[0])

            # Calculate the "real score" by
            #    (lowest score in group) * [ (group size)^aggressiveness - 1].
            # The reason we subtract one is that we never want to have a group of
            # size 1.
        
        if type(vecteur_preference) == np.ndarray:
            vocab_group = np.array([nlp.vocab.get_vector(str(pos[idx])) for idx in [tpl[1] for tpl in group]])
            group_center = np.mean(vocab_group, axis=0)
            clue_nlp = nlp.vocab.get_vector(str(clue))
            direction = clue_nlp - group_center
            direction = direction / np.linalg.norm(direction)
            # Cosine similarity normalized to [0, 1]
            direction_coef = (pairwise.cosine_similarity(direction.reshape(1, -1), vecteur_preference.reshape(1, -1))[0][0] + 1) /2     
        else:
            direction_coef = 1
            
        groups_score.append(direction_coef*((np.min(group_score) - (boundary/len(group))) * (len(group)**agg - 0.99)))
        groups.append([tpl[1] for tpl in group])
    
    real_score = max(groups_score)
    word_indices = groups[np.argmax(groups_score)]  
    
    return real_score, word_indices

In [19]:
def get_clue(pos_words, neg_words, neu_words, assassin_word, danger_coeff=3, agg=0.1, given_indices=[], vecteur_preference=None):
    
    best_clue, best_score, words_to_guess = None, -1 , None
    
    df = get_score_df(pos_words, neg_words, neu_words, assassin_word, given_indices, danger_coeff)
    
    for candidate in df.index:
        real_score, word_ind = calculate_clue_score(candidate, pos_words, agg, df, neu_words, vecteur_preference)
        if real_score > best_score:
            best_clue = candidate
            best_score = real_score
            words_to_guess = np.array(pos_words)[word_ind]
            #print(f"clue: {best_clue}, words: {words_to_guess}, score:{best_score}")
    

    return best_clue, best_score, words_to_guess


In [19]:
vecteur_preference = np.load('preference.npy')

In [15]:
#Build plateau
class Plateau():

    #constructor
    def __init__(self, B, R, N, A, status):

        #list of words
        self.B = list(B)
        self.R = list(R)
        self.N = list(N)
        self.A = A

        #Status = 1. If 0, then game is over
        self.status = status

        #all words
        lst = np.concatenate([B, R, N], axis=0)
        words = np.concatenate([lst, [A]])
        random.shuffle(words)
        self.words = words
        


    #methods
    def update_status(self):
        """
        This method uses the inputs (words and their associated team to build new attributes: list of words for each team.
        This should be used after every update of the Plateau
        """

        #If one of these is empty: game is over. Status becomes 0, it ends the game
        if min([len(self.B), len(self.R), len(self.A)]) == 0:
            self.status = 0
            print("Game is over")

        #print(f"There is {len(self.B)} words for team BLUE\n")
        #print(f"There is {len(self.R)} words for team RED\n")
        #print(f"There is {len(self.N)} neutral words\n")
        #print(f"There is {len(self.A)} assasin\n")

        return None

    def remove_word(self, chosen_word):
        """
        This function takes a word that the user wants to remove. It prompts a message and then proceeds to remove it.
        """
        #check if in list
        if chosen_word == "next":
            print("\nYou decided to pass.\n")
            return None
        
        if chosen_word not in self.words:
            return self.remove_word(chosen_word=input("Choose another one, this one is not in the board\n>>"))
        
        else:
            self.words = np.where(self.words==chosen_word, "X", self.words) 
        #print(f" \nYou decided to remove {chosen_word}\n")

        #remove
        if chosen_word in self.B:
            print('It was a BLUE word!\n')
            self.B.remove(chosen_word)
            self.update_status()

        if chosen_word in self.R:
            print('It was a RED word!\n')
            self.R.remove(chosen_word)
            self.update_status()

        if chosen_word in self.N:
            print('It was a NEUTRAL word!\n')
            self.N.remove(chosen_word)
            self.update_status()

        if chosen_word in self.A:
            print('It was the ASSASSIN word!\n')
            self.A = []
            self.update_status()

        return None


In [16]:
def generate_board():
    
    rs = np.array(random.sample(words, 25))
    print("BOARD\n")
    print(rs.reshape(5,5))
    
    np.random.shuffle(rs)
  
    B = rs[:8]
    R = rs[8:17]
    N = rs[17:24]
    A = rs[24]
    
    return B, R, N, A

In [49]:
B, R, N, A = generate_board()
print(B)
print(R)
print(N)
print(A)

BOARD

[['bouton' 'cercle' 'tuyau' 'bulle' 'ombre']
 ['chine' 'point' 'jacques' 'genou' 'chat']
 ['résistance' 'jupiter' 'musique' 'culotte' 'université']
 ['vent' 'cabine' 'asie' 'orange' 'lune']
 ['brise' 'rayé' 'amérique' 'œil' 'sapin']]
['asie' 'cercle' 'rayé' 'point' 'musique' 'chat' 'orange' 'bulle']
['résistance' 'amérique' 'jacques' 'vent' 'chine' 'bouton' 'culotte'
 'sapin' 'genou']
['lune' 'tuyau' 'brise' 'œil' 'université' 'jupiter' 'ombre']
cabine


In [50]:
get_clue(B, R, N, A, danger_coeff=3, agg=0.1, given_indices=[], vecteur_preference=None)

('théâtre',
 0.09796416428187565,
 array(['point', 'musique', 'orange'], dtype='<U10'))

In [51]:
get_clue(B, R, N, A, danger_coeff=3, agg=0.1, given_indices=[], vecteur_preference=vecteur_preference)

('sphère',
 0.046298947179855825,
 array(['cercle', 'point', 'bulle'], dtype='<U10'))

In [27]:
#Lauch game!
B, R, N, A = generate_board()
plateau = Plateau(B, R, N, A, 1)
i = 0

given_indices = []

INDICES = {}

NB_TOURS = 0
while plateau.status == 1:
    i = (-1)*i + 1
    team_name = ["BLUE", "RED"][i]
    NB_TOURS += 1
    plateau.update_status()
    team = [plateau.B, plateau.R][i]
    opponent = [plateau.R, plateau.B][i]
    print("__________________")
    print(f"This turn, you are {team_name}")
    print(f"Words remaining for team: {len(team)}")
    print(f"Words remaining for opponent: {len(opponent)}\n")
    
    if i == 1:
        clue, score, group = get_clue(team, opponent, plateau.N, plateau.A, given_indices=given_indices)
    if i == 0:
        clue, score, group = get_clue(team, opponent, plateau.N, plateau.A, given_indices=given_indices)
    
    INDICES[clue] = list(group)
    given_indices.append(clue)
    print(f"The given clue: {clue}")
    print(f"Words to find: {len(group)}\n")
    for k in range(len(group)):
        print(plateau.words.reshape(5,5))
        w = input("Please chose a word to remove\n>>")
        if w in opponent or w in plateau.N:
            plateau.remove_word(w)
            print("End of turn")
            break
        
        plateau.remove_word(w)
        plateau.update_status()
        if plateau.status == 0:
            break
    
print("____________________\n")
print("____________________\n")
print("____________________\n")
print(f"NB TOURS: {NB_TOURS}")

for c in INDICES:
    print(f"Clue: {c}, words to get: {INDICES[c]}")

BOARD

[['twitter' 'coccinelle' 'bravo' 'zen' 'diamant']
 ['jambon' 'ski' 'napoléon' 'cabine' 'tube']
 ['épine' 'chaîne' 'dormir' 'poche' 'sourd']
 ['pirate' 'chemise' 'chocolat' 'trône' 'lune']
 ['volume' 'meuble' 'courrier' 'machine' 'licorne']]
__________________
This turn, you are RED
Words remaining for team: 9
Words remaining for opponent: 8

The given clue: chaise
Words to find: 3

[['licorne' 'tube' 'napoléon' 'diamant' 'chaîne']
 ['chocolat' 'poche' 'coccinelle' 'twitter' 'meuble']
 ['jambon' 'lune' 'bravo' 'ski' 'courrier']
 ['sourd' 'trône' 'chemise' 'cabine' 'volume']
 ['zen' 'dormir' 'épine' 'machine' 'pirate']]
Please chose a word to remove
>>meuble
It was a RED word!

[['licorne' 'tube' 'napoléon' 'diamant' 'chaîne']
 ['chocolat' 'poche' 'coccinelle' 'twitter' 'X']
 ['jambon' 'lune' 'bravo' 'ski' 'courrier']
 ['sourd' 'trône' 'chemise' 'cabine' 'volume']
 ['zen' 'dormir' 'épine' 'machine' 'pirate']]
Please chose a word to remove
>>trône
It was a RED word!

[['licorne' 't

KeyboardInterrupt: Interrupted by user

remove already given clues

## Test AI Fast

In [38]:
B, R, N, A = generate_board()

BOARD

[['dragon' 'roue' 'pâte' 'atchoum' 'mer']
 ['cartouche' 'gel' 'heureux' 'coude' 'nage']
 ['bretagne' 'grippe' 'fenêtre' 'cabinet' 'gros']
 ['toast' 'bouteille' 'aile' 'action' 'politique']
 ['solution' 'sang' 'cercle' 'triangle' 'vaisselle']]


In [39]:
B

array(['action', 'grippe', 'triangle', 'solution', 'fenêtre', 'gel',
       'bouteille', 'cercle'], dtype='<U9')

In [47]:
pos_words, neg_words, neu_words, assassin_word = B, R, N, A
best_clue, best_score, best_g = get_clue(pos_words, neg_words, neu_words, assassin_word, danger_coeff=3, agg=0.1, given_indices=[])
print(best_clue)
print(best_score)
print(best_g)

contour
24.71202135107106
['fenêtre' 'triangle' 'solution' 'gel' 'action']


## Evaluation AI NLP

In [26]:
def generate_board_var(board_size):
    
    if board_size == 25:
        rs = np.array(random.sample(words, 25))    
        np.random.shuffle(rs)  
        J1_words = rs[:8]
        J2_words = rs[8:17]
        N_words = rs[17:24]
        A_words = rs[24]
    
    elif board_size == 20:
        rs = np.array(random.sample(words, 20))   
        np.random.shuffle(rs)  
        J1_words = rs[:6]
        J2_words = rs[6:12]
        N_words = rs[12:19]
        A_words = rs[19]
    
    elif board_size == 15:
        rs = np.array(random.sample(words, 15))    
        np.random.shuffle(rs)  
        J1_words = rs[:4]
        J2_words = rs[4:8]
        N_words = rs[8:14]
        A_words = rs[14]
    
    elif board_size == 10:
        rs = np.array(random.sample(words, 10)) 
        np.random.shuffle(rs)  
        J1_words = rs[:3]
        J2_words = rs[3:6]
        N_words = rs[6:9]
        A_words = rs[9]
    
    return J1_words, J2_words, N_words, A_words

In [24]:
def evaluation_nlp(n_iterations=1):
    
    n_J1, n_J2, n_N, n_A = 0, 0, 0, 0
    
    for i in range(n_iterations):
        
        list_sizes = [25, 20, 15, 10]
        for board_size in list_sizes:
            pos_words, neg_words, neu_words, assassin_word = generate_board_var(board_size)
            plateau = Plateau(pos_words, neg_words, neu_words, assassin_word, 1)
            
            clue, score, group = get_clue(pos_words, neg_words, neu_words, assassin_word, given_indices=[])
            
            print(f"The given clue: {clue}")
            print(f"Words to find: {len(group)}\n")
            for k in range(len(group)):
                print(plateau.words.reshape(board_size//5,5))
                w = input("Please chose a word to remove\n>>")
                while w not in plateau.words:
                    w = input("Please chose a word from the board\n>>")
                if w == "next":
                    print("You decided to pass.")
                    break
                elif w in plateau.R:
                    print("Negative word")
                    n_J2 += 1
                    break
                elif w in plateau.N:
                    print("Neutral word")
                    n_N += 1
                    break
                elif w in plateau.A:
                    print("Assassin word")
                    n_A += 1
                    break
                else: 
                    print("Positive word")
                    n_J1 += 1        
                    plateau.remove_word(w)
                    plateau.update_status()
                    if plateau.status == 0:
                        break
    
    n_J1 /= 4*n_iterations
    n_J2 /= 4*n_iterations
    n_N /= 4*n_iterations
    n_A /= 4*n_iterations
            
    return n_J1, n_J2, n_N, n_A        

In [33]:
n_J1, n_J2, n_N, n_A = evaluation_nlp()

The given clue: spectacle
Words to find: 4

[['salle' 'titre' 'angleterre' 'tournoi' 'tokyo']
 ['serrer' 'inviter' 'pierre' 'présent' 'explosion']
 ['manège' 'poste' 'mou' 'bateau' 'voile']
 ['élève' 'mousse' 'fête' 'public' 'docteur']
 ['star' 'paris' 'basket' 'fauteuil' 'plante']]
Please chose a word to remove
>>salle
Positive word
It was a BLUE word!

[['X' 'titre' 'angleterre' 'tournoi' 'tokyo']
 ['serrer' 'inviter' 'pierre' 'présent' 'explosion']
 ['manège' 'poste' 'mou' 'bateau' 'voile']
 ['élève' 'mousse' 'fête' 'public' 'docteur']
 ['star' 'paris' 'basket' 'fauteuil' 'plante']]
Please chose a word to remove
>>public
Positive word
It was a BLUE word!

[['X' 'titre' 'angleterre' 'tournoi' 'tokyo']
 ['serrer' 'inviter' 'pierre' 'présent' 'explosion']
 ['manège' 'poste' 'mou' 'bateau' 'voile']
 ['élève' 'mousse' 'fête' 'X' 'docteur']
 ['star' 'paris' 'basket' 'fauteuil' 'plante']]
Please chose a word to remove
>>star
Positive word
It was a BLUE word!

[['X' 'titre' 'angleterre' 'to

In [34]:
def agregation_NLP(n_J1, n_J2, n_N, n_A):
    print("IA score: ", n_J1 - n_J2 - 3*n_A)
    return n_J1 - n_J2 - 3*n_A

In [35]:
agregation_NLP(n_J1, n_J2, n_N, n_A)

IA score:  0.75


0.75

## Evaluation AI In-Game

In [20]:
def evaluation_ingame(n_iterations=2):
    
    dict_scores = {'n_J1': [], 'n_J2': [], 'n_A': [], 'T': []}
    
    for i in range(1, n_iterations+1):
        
        print(f"Game {i}/{n_iterations}")
        print("You play the RED team")
        n_J1, n_J2, n_A, T = 0, 0, 0, 0
    
        B, R, N, A = generate_board()
        plateau = Plateau(B, R, N, A, 1)
        turn = 1

        given_indices = []
        
        while plateau.status == 1:

            plateau.update_status()
            team = plateau.R
            opponent = plateau.B
            
            print(f"___Turn {turn}____")
            print(f"Words remaining for team: {len(team)}")
            print(f"Words remaining for opponent: {len(opponent)}\n")
            
            # Turn of J1 (Red)
            clue, score, group = get_clue(team, opponent, plateau.N, plateau.A, given_indices=given_indices)
    
            given_indices.append(clue)
            print(f"The given clue: {clue}")
            print(f"Words to find: {len(group)}\n")
        
            for k in range(len(group)):
            
                print(plateau.words.reshape(5,5))
                w = input("Please chose a word to remove\n>>")
                
                if w in opponent:
                    n_J2 += 1
                    plateau.remove_word(w)
                    print("End of turn")
                    break
                    
                if w in plateau.N:
                    plateau.remove_word(w)
                    print("End of turn")
                    break
        
                if w in plateau.A:
                    n_A += 1
                    plateau.remove_word(w)
                    print("End of turn")
                    break
                    
                if w in plateau.R:
                    n_J1 += 1
                    plateau.remove_word(w)
                    
            if plateau.status == 0:
                break
                    
            # Turn of J2 (Blue)
            if turn % 2 == 1 :
                remove_J2 = random.choice(plateau.B)
                print("Word removed for J2: ", remove_J2)
                plateau.remove_word(remove_J2)
                if plateau.status == 0:
                    break
                remove_J2 = random.choice(plateau.B)
                print("Word removed for J2: ", remove_J2)
                plateau.remove_word(remove_J2)
                if plateau.status == 0:
                    break
                if len(plateau.N) > 0:
                    remove_N = random.choice(plateau.N)
                    print("Word removed for N: ", remove_N)
                    plateau.remove_word(remove_N)
                    
            if turn % 2 == 0 :
                remove_J2 = random.choice(plateau.B)
                print("Word removed for J2: ", remove_J2)
                plateau.remove_word(remove_J2)
                if plateau.status == 0:
                    break
                remove_J1 = random.choice(plateau.R)
                print("Word removed for J1: ", remove_J1)
                plateau.remove_word(remove_J1)
                if plateau.status == 0:
                    break                  
                    
            turn += 1     
        
        print(f"Score for this game: n_J1 = {n_J1}, n_J2 = {n_J2}, n_A = {n_A}, T = {turn}")
        
        dict_scores['n_J1'].append(n_J1)
        dict_scores['n_J2'].append(n_J2) 
        dict_scores['n_A'].append(n_A)  
        dict_scores['T'].append(turn)
                    
    return dict_scores

In [21]:
dict_scores = evaluation_ingame(n_iterations=5)

Game 1/5
You play the RED team
BOARD

[['achat' 'voiture' 'jean' 'jardin' 'noir']
 ['balai' 'orange' 'lapin' 'coq' 'côte']
 ['personne' 'poule' 'rap' 'formule' 'robot']
 ['uniforme' 'ronde' 'confort' 'tête' 'coup']
 ['cadre' 'pompe' 'crochet' 'papa' 'chêne']]
___Turn 1____
Words remaining for team: 9
Words remaining for opponent: 8

The given clue: gauche
Words to find: 3

[['chêne' 'côte' 'personne' 'coup' 'tête']
 ['poule' 'robot' 'uniforme' 'voiture' 'formule']
 ['papa' 'cadre' 'ronde' 'balai' 'pompe']
 ['achat' 'jardin' 'noir' 'confort' 'crochet']
 ['rap' 'coq' 'orange' 'jean' 'lapin']]
Please chose a word to remove
>>côte
It was a BLUE word!

End of turn
Word removed for J2:  confort
It was a BLUE word!

Word removed for J2:  orange
It was a BLUE word!

Word removed for N:  papa
It was a NEUTRAL word!

___Turn 2____
Words remaining for team: 9
Words remaining for opponent: 5

The given clue: chasseur
Words to find: 3

[['chêne' 'X' 'personne' 'coup' 'tête']
 ['poule' 'robot' 'unif

The given clue: McLaren
Words to find: 2

[['twitter' 'sec' 'X' 'X' 'cochon']
 ['courant' 'sport' 'télévision' 'centre' 'boulet']
 ['culotte' 'X' 'marche' 'moteur' 'muscle']
 ['licorne' 'cartouche' 'saison' 'tigre' 'prier']
 ['eau' 'mémoire' 'X' 'sable' 'X']]
Please chose a word to remove
>>moteur
It was a RED word!

[['twitter' 'sec' 'X' 'X' 'cochon']
 ['courant' 'sport' 'télévision' 'centre' 'boulet']
 ['culotte' 'X' 'marche' 'X' 'muscle']
 ['licorne' 'cartouche' 'saison' 'tigre' 'prier']
 ['eau' 'mémoire' 'X' 'sable' 'X']]
Please chose a word to remove
>>sport
It was a BLUE word!

End of turn
Word removed for J2:  prier
It was a BLUE word!

Word removed for J1:  eau
It was a RED word!

___Turn 3____
Words remaining for team: 6
Words remaining for opponent: 4

The given clue: maillot
Words to find: 2

[['twitter' 'sec' 'X' 'X' 'cochon']
 ['courant' 'X' 'télévision' 'centre' 'boulet']
 ['culotte' 'X' 'marche' 'X' 'muscle']
 ['licorne' 'cartouche' 'saison' 'tigre' 'X']
 ['X' 'mémoire' 

The given clue: sein
Words to find: 2

[['X' 'X' 'garde' 'X' 'X']
 ['X' 'fourmi' 'poisson' 'X' 'vingt']
 ['cabane' 'sport' 'X' 'X' 'ami']
 ['X' 'X' 'foyer' 'X' 'nœud']
 ['épine' 'X' 'X' 'X' 'kiwi']]
Please chose a word to remove
>>kiwi
It was a BLUE word!

End of turn
Word removed for J2:  poisson
It was a BLUE word!

Game is over
Score for this game: n_J1 = 3, n_J2 = 2, n_A = 0, T = 4


In [22]:
dict_scores

{'n_J1': [7, 5, 5, 5, 3],
 'n_J2': [1, 0, 1, 2, 2],
 'n_A': [0, 0, 1, 0, 0],
 'T': [5, 5, 4, 4, 4]}

In [23]:
def agregation_ingame(dict_scores):
    score, iters = 0, len(dict_scores['n_J1'])
    for i in range(iters):
        score += ((dict_scores['n_J1'][i] - dict_scores['n_J2'][i] - 2*dict_scores['n_A'][i])/dict_scores['T'][i])
    score /= iters
    print("IA score: ", score)
    return score

In [24]:
agregation_ingame(dict_scores)

IA score:  0.74


0.74

## Apprentissage vecteur de préférence

In [80]:
def apprentissage(n_iterations=20):
    
    vecteur_preference = np.zeros(300)
    n_J1, n_J2, n_N, n_A = 0, 0, 0, 0
    
    for i in range(1, n_iterations+1):
        
        board_size = 25
        pos_words, neg_words, neu_words, assassin_word = generate_board_var(board_size)
        plateau = Plateau(pos_words, neg_words, neu_words, assassin_word, 1)
            
        clue, score, group = get_clue(pos_words, neg_words, neu_words, assassin_word, given_indices=[])
        vocab_group = np.array([nlp.vocab.get_vector(str(word)) for word in group])
        group_center = np.mean(vocab_group, axis=0)
        clue_nlp = nlp.vocab.get_vector(str(clue))
        direction = clue_nlp - group_center
        direction = direction / np.linalg.norm(direction)
        
        n_J1, n_J2 = 0, 0
            
        print(f"The given clue: {clue}")
        print(f"Words to find: {len(group)}\n")
        
        for k in range(len(group)):
            print(plateau.words.reshape(board_size//5,5))
            w = input("Please chose a word to remove\n>>")
            while w not in plateau.words:
                w = input("Please chose a word from the board\n>>")
            if w in plateau.R:
                print("Negative word")
                n_J2 += 1
                break
            elif w in plateau.N:
                print("Neutral word")
                n_J2 += 1
                break
            elif w in plateau.A:
                print("Assassin word")
                n_J2 += 1
                break
            else: 
                print("Positive word")
                n_J1 += 1        
                plateau.remove_word(w)
                plateau.update_status()
                if plateau.status == 0:
                    break
                    
        score = 2 * (n_J1 / (n_J1 + n_J2)) - 1
        vecteur_run = score * direction
        vecteur_preference += 1/n_iterations * vecteur_run
    
    vecteur_preference = vecteur_preference / np.linalg.norm(vecteur_preference)
    return vecteur_preference

In [81]:
vecteur_preference = apprentissage()

The given clue: d'araignée
Words to find: 2

[['coq' 'olympique' 'canine' 'alphabet' 'charme']
 ['bob' 'poubelle' 'rond' 'château' 'barbe']
 ['prise' 'araignée' 'ampoule' 'mur' 'mer']
 ['tuyau' 'tissu' 'champion' 'roulette' 'danse']
 ['numéro' 'dictionnaire' 'poussière' 'appétit' 'air']]
Please chose a word to remove
>>araignée
Positive word
It was a BLUE word!

[['coq' 'olympique' 'canine' 'alphabet' 'charme']
 ['bob' 'poubelle' 'rond' 'château' 'barbe']
 ['prise' 'X' 'ampoule' 'mur' 'mer']
 ['tuyau' 'tissu' 'champion' 'roulette' 'danse']
 ['numéro' 'dictionnaire' 'poussière' 'appétit' 'air']]
Please chose a word to remove
>>poubelle
Neutral word
The given clue: presqu'île
Words to find: 2

[['poubelle' 'blanc' 'troupeau' 'thé' 'balance']
 ['poêle' 'marque' 'patin' 'cause' 'bête']
 ['mer' 'vision' 'prêt' 'baie' 'but']
 ['vase' 'afrique' 'éléphant' 'nuage' 'courrier']
 ['privé' 'pince' 'génie' 'rome' 'toujours']]
Please chose a word to remove
>>mer
Positive word
It was a BLUE word!

[[

Please chose a word to remove
>>sale
Positive word
It was a BLUE word!

The given clue: Vichy
Words to find: 2

[['sport' 'puzzle' 'boisson' 'internet' 'danse']
 ['bac' 'soeur' 'trait' 'iris' 'accident']
 ['pensée' 'marais' 'physique' 'reptile' 'œuf']
 ['jeu' 'station' 'sexe' 'vingt' 'gomme']
 ['éléphant' 'tuile' 'moulin' 'penser' 'lait']]
Please chose a word to remove
>>boisson
Positive word
It was a BLUE word!

[['sport' 'puzzle' 'X' 'internet' 'danse']
 ['bac' 'soeur' 'trait' 'iris' 'accident']
 ['pensée' 'marais' 'physique' 'reptile' 'œuf']
 ['jeu' 'station' 'sexe' 'vingt' 'gomme']
 ['éléphant' 'tuile' 'moulin' 'penser' 'lait']]
Please chose a word to remove
>>marais
Negative word
The given clue: amoureux
Words to find: 3

[['poète' 'gazon' 'vite' 'mariée' 'australie']
 ['trésor' 'loup' 'chant' 'heureux' 'opéra']
 ['indice' 'chou' 'papier' 'ligne' 'pince']
 ['fin' 'culotte' 'oiseau' 'farce' 'basket']
 ['discours' 'fuite' 'chaud' 'goutte' 'tokyo']]
Please chose a word to remove
>>ma

In [87]:
np.save('preference', vecteur_preference)

## Utilisation vecteur de préférence

In [168]:
def calculate_clue_score(clue, pos, agg, df, neu, vecteur_preference):
    scores = df.loc[df.index == clue]
    if len(neu) == 0:
        if scores[["neg", "ass"]].max(axis=1)[0] == 1:
            return -1, None
    else:
        if scores[["neg", "neu", "ass"]].max(axis=1)[0] == 1:
            return -1, None
    # Order scores by highest to lowest (highest is best)
    pos_scores = scores.iloc[:, :len(pos)].values[0]
    boundary = np.max(scores.iloc[:,len(pos):], axis=1).values[0]
    i = 0
    
    # Order scores by lowest to highest inner product with the clue.
    ss = sorted((s, i) for i, s in enumerate(pos_scores))
    groups = []
    groups_score = []
    for i in range(len(pos_scores)):
        group = ss[-(i+1):]
#         print('GROUPE: ', group)
        group_score = []
        for tpl in group:
            group_score.append(tpl[0])

            # Calculate the "real score" by
            #    (lowest score in group) * [ (group size)^aggressiveness - 1].
            # The reason we subtract one is that we never want to have a group of
            # size 1.
        
        vocab_group = np.array([nlp.vocab.get_vector(str(pos[idx])) for idx in [tpl[1] for tpl in group]])
        group_center = np.mean(vocab_group, axis=0)
        clue_nlp = nlp.vocab.get_vector(str(clue))
        direction = clue_nlp - group_center
        direction = direction / np.linalg.norm(direction)
        direction_coef = (pairwise.cosine_similarity(direction.reshape(1, -1), vecteur_preference.reshape(1, -1))[0][0] + 1) /2
        
        groups_score.append(direction_coef*((np.min(group_score) - (boundary/len(group))) * (len(group)**agg - 0.99)))
        groups.append([tpl[1] for tpl in group])
    
    real_score = max(groups_score)
    word_indices = groups[np.argmax(groups_score)]  
    
    return real_score, word_indices